In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
import shutil
from sklearn.model_selection import train_test_split
import urllib.request
import tarfile

# Enable mixed precision training for T4 GPU
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# Directory where the dataset is stored
train_dir = "/content/drive/MyDrive/Fish_detection_project/BD_Freshwater_Fish_An_Image_Dataset_from_Bangladesh_for_AI_Powered_Automatic_Fish_Species_Classification_and_Detection_toward_Smart_Aquaculture/BD_Freshwater_Fish"

# Verify dataset structure
classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
print(f"Found {len(classes)} species: {classes}")
if len(classes) != 12:
    print("Warning: Expected 12 species folders, please check the directory structure.")

# Define paths for train, validation, and test directories
base_dir = "/content/drive/MyDrive/Fish_detection_project/split_dataset"
train_path = os.path.join(base_dir, 'train')
val_path = os.path.join(base_dir, 'validation')
test_path = os.path.join(base_dir, 'test')

# Create directories if they don't exist
for path in [train_path, val_path, test_path]:
    os.makedirs(path, exist_ok=True)

# Function to split dataset
def split_dataset(source_dir, train_path, val_path, test_path, train_size=0.7, val_size=0.15):
    class_counts = {}
    for class_name in classes:
        os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_path, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

        class_dir = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        print(f"Class {class_name}: {len(images)} images")
        class_counts[class_name] = len(images)

        train_val_images, test_images = train_test_split(images, test_size=0.15, random_state=42)
        train_images, val_images = train_test_split(train_val_images, test_size=val_size/(train_size + val_size), random_state=42)

        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(train_path, class_name, img))
        for img in val_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(val_path, class_name, img))
        for img in test_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(test_path, class_name, img))
        print(f"Class {class_name}: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test")

    return class_counts

# Split the dataset and get class counts for weighting
class_counts = split_dataset(train_dir, train_path, val_path, test_path)

# Compute class weights to handle imbalance
total_samples = sum(class_counts.values())
class_weights = {i: total_samples / (12 * class_counts[cls]) for i, cls in enumerate(classes)}
print("Class weights:", class_weights)

# Define image parameters
IMG_HEIGHT, IMG_WIDTH = 128, 128  # Your specified target size
BATCH_SIZE = 16  # Your specified batch size, suitable for T4 GPU

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Your specified augmentation
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Print class indices
print("Class indices:", train_generator.class_indices)

# Manually download MobileNetV2 weights
weights_url = "https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5"
weights_path = "/content/mobilenet_v2_weights.h5"

print("Downloading MobileNetV2 weights...")
urllib.request.urlretrieve(weights_url, weights_path)

# Build the model using MobileNetV2 with custom top
base_model = MobileNetV2(weights=weights_path, include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze the base model

model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(12, activation='softmax', dtype='float32')  # 12 species, float32 for mixed precision
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model with class weights
try:
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=30,
        steps_per_epoch=len(train_generator),
        validation_steps=len(val_generator),
        class_weight=class_weights
    )
except Exception as e:
    print("Error during model training:", e)

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model
model.save('/content/drive/MyDrive/Fish_detection_project/fish_classification_model.keras')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Fish_detection_project/BD_Freshwater_Fish_An_Image_Dataset_from_Bangladesh_for_AI_Powered_Automatic_Fish_Species_Classification_and_Detection_toward_Smart_Aquaculture/BD_Freshwater_Fish'